# Import Libraries

In [1]:
from swarmintelligence import WhaleOptimizationAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/WOA/optimal_params.pkl', 'rb'))
optimal_params

{'n_whales': 25, 'maxIteration': 100, 'constanta': 5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
930.7811479817464  |  2436.3378240861466  |  1787.9236630530743
930.7047355196313  |  2436.3584799702053  |  1787.2100683323306
930.7811479817464  |  2436.3584799702053  |  1787.8215619288128
930.7683528911109  |  2436.3584799702053  |  1787.8888109611032
930.7811479817464  |  2436.15956953858  |  1787.9236630530743
930.6625935315783  |  2436.3584799702053  |  1787.9236630530743
930.7013606497757  |  2436.3584799702053  |  1787.8045911666616
930.7811479817464  |  2436.3573686548248  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.3941143583256
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.8210861061968
930.7811479817464  |  2436.3584799702053  |  1787.8210861061968
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2435.779131213827  |  1787.9236630530743
930.556

1004.9139322682681  |  2393.647254097627  |  2181.5289652445044
1004.9139322682681  |  2393.6040262694814  |  2181.8670821363294
1004.9139322682681  |  2393.6040262694814  |  2181.813810795207
1004.9139322682681  |  2393.647254097627  |  2181.782175541647
1004.7658946943336  |  2393.408017759868  |  2181.813810795207
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.4563101761905
1004.9139322682681  |  2393.647254097627  |  2180.920452728812
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.7658946943336  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.782175541647
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.7888338820939  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.758352748785
1004.9139322682681  |  2393.647254097627  |  

2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.2405617700765  |  2947.3859621212478  |  2947.2023498081107
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.1042090279807
2947.109394071251  |  2947.109394071251  |  2947.3859621212478
2947.2405617700765  |  2947.3859621212478  |  2947.3859621212478
2947.2405617700765  |  2947.3859621212478  |  2947.109394071251
2947.109394071251  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.109394071251  |  2947.3859621212478
2947.2405617700765  |  2947.368844954497  |  2522.922743523481
2947.3859621212478  |  2947.3859621212478  |  2947.2405617700765
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.368844954497  |  2947.3859621212478
2947.3859621212478  |  2947.10939

3359.743500726603  |  2004.6997362709596  |  2640.869259570156
3359.537929144166  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8307253277617  |  2004.6997362709596  |  2640.825119524543
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8307253277617  |  2004.4553143805483  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8307253277617  |  2004.4945853679887  |  2640.869259570156
3359.8307253277617  |  2004.6997362709596  |  2640.1531263224942
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.537929144166  |  2004.4553143805483  |  2640.869259570156
3359.8307253277617  |  2004.4061790804808  |  2640.869259570156
3359.4664216430997  |  1691.9444262646568  |  2640.869259570156
3359.8652306428266  |  2004.4945853679887  |  2640.825119524543
3359.8652306428266  |  1692.4170825213273 

1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.1370985034855  |  5203.927265675553  |  1759.1122795776703
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.0824237250008  |  5203.927265675553  |  1759.2175161838823
1656.1370985034855  |  5203.927265675553  |  1759.1122795776703
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.0997728044963  |  5203.7878061953315  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.1359980441362
1656.1590831079752  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1758.9276274618476
1169.5603425842705  |  5203.92726567555

5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.5693811079145  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.730962403627
5640.428003063228  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.585936244663  |  1589.730962403627
5640.302076522936  |  5636.585936244663  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.418808324037  |  5636.585936244663  |  1589.872409058858
5640.428003063228  |  5636.585936244663  |  1589.872409058858
5640.428003063228  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.2376919682213
5640.418808324037  |  5636.585936244663  |  1589.872409058858
5640.1329366354175  |  5636.5693811079145  |  1589.872409058858
5640.21901646257  |  5636.5693811079145  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.8493731099352
564

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[121, 178]","[94, 166]","[125, 182]",930.740033,2436.307211,1787.841193,...,"[[125, 255, 255, 255, 255, 255, 255, 255, 255,...","[930.7811479817464, 930.7047355196313, 930.781...","[2436.3378240861466, 2436.3584799702053, 2436....","[1787.9236630530743, 1787.2100683323306, 1787....","[1.0624656677246094, 0.9476759433746338, 0.986...","[2296.0388272603354, 2310.284954071045, 2300.5...","[47.916999355764496, 48.06542368554599, 47.964...","[14.521011329249303, 14.49414811128002, 14.512...","[0.7808390678281049, 0.7819625051247647, 0.781...","[0.9409589634662838, 0.9409256896535507, 0.940..."
1,AirplaneF16.tiff,3,otsu,max,"[105, 156, 195]","[64, 129, 184]","[121, 172, 201]",970.807184,2517.659006,1856.241911,...,"[[121, 255, 201, 255, 255, 201, 201, 201, 201,...","[967.7826905710319, 970.9357686570892, 971.116...","[2533.246043083906, 2534.072236968903, 2533.84...","[1856.4464905049515, 1856.4194057805355, 1856....","[4.060472726821899, 3.053114414215088, 3.05702...","[1570.8148829142253, 1591.8677202860515, 1582....","[39.633507073109556, 39.8982170063532, 39.7784...","[16.1695535346376, 16.11173384612408, 16.13785...","[0.7332509394673451, 0.738893315636063, 0.7345...","[0.9590481233773241, 0.9581388748339702, 0.958..."
2,AirplaneF16.tiff,4,otsu,max,"[81, 125, 168, 199]","[59, 112, 162, 201]","[110, 150, 180, 204]",994.305679,2581.846514,1901.257102,...,"[[110, 204, 204, 255, 204, 204, 204, 204, 204,...","[994.1987450446633, 994.5828009412995, 994.362...","[2587.569390438851, 2587.247794576184, 2587.49...","[1901.4583768781156, 1901.8798605160753, 1901....","[6.107685804367065, 4.638679027557373, 5.16942...","[1184.1273384094238, 1161.6300786336262, 1202....","[34.41115136709936, 34.082694709098725, 34.673...","[17.396819529109976, 17.480125119751474, 17.33...","[0.719621682623988, 0.7162854667079372, 0.7198...","[0.9690958671138267, 0.9696356235104008, 0.968..."
3,AirplaneF16.tiff,5,otsu,max,"[69, 108, 143, 177, 202]","[32, 80, 127, 171, 203]","[105, 142, 169, 191, 207]",1005.242664,2600.575739,1921.333185,...,"[[105, 207, 207, 207, 207, 207, 207, 207, 207,...","[1004.9448404008972, 1006.62369363615, 997.343...","[2586.912337934203, 2587.4745328451127, 2618.8...","[1922.939330889983, 1922.145585132126, 1913.64...","[4.259164810180664, 5.208019733428955, 3.35508...","[961.3007647196451, 975.8420562744141, 1043.31...","[31.004850664366135, 31.23847077362165, 32.300...","[18.302210730722294, 18.237008297168742, 17.94...","[0.7279375596439555, 0.7232094662385563, 0.722...","[0.9744989075142958, 0.9741430682823168, 0.973..."
4,Lena.png,2,otsu,max,"[136, 195]","[78, 146]","[96, 141]",1004.875085,2393.620449,2181.737453,...,"[[141, 141, 141, 141, 141, 141, 141, 141, 141,...","[1004.9139322682681, 1004.9139322682681, 1004....","[2393.647254097627, 2393.6040262694814, 2393.6...","[2181.5289652445044, 2181.8670821363294, 2181....","[3.880260467529297, 3.9055016040802, 3.8449432...","[1866.2016716003418, 1867.6266797383626, 1870....","[43.199556381985474, 43.21604655377864, 43.250...","[15.421217867679474, 15.417902915074137, 15.41...","[0.6398055118597031, 0.6386347349228899, 0.639...","[0.8954977946512496, 0.8955093562237663, 0.895..."
5,Lena.png,3,otsu,max,"[125, 173, 210]","[56, 106, 159]","[82, 110, 147]",1069.858683,2597.722691,2270.202643,...,"[[147, 147, 147, 147, 147, 147, 147, 147, 147,...","[1069.0067992342251, 1069.4829033163626, 1070....","[2597.917277382854, 2597.9684233194057, 2597.9...","[2270.088023340101, 2269.7571605012254, 2270.2...","[3.871539354324341, 3.1151716709136963, 3.0609...","[1180.9528643290203, 1187.2970136006672, 1179....","[34.364994752349666, 34.45717651811691, 34.347...","[17.408477970142002, 17.38520985441316, 17.412...","[0.6972316

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/WOA/WOA_otsu_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/WOA/WOA_otsu_RGB_dict.pkl', 'wb'))